In [163]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

In [164]:
class NaiveBayesClassifier():
    def calc_prior(self, features, target):
        cop=self
        cop.prior=(features.groupby(target).apply(supporter)/self.rows).to_numpy()
        self.prior=cop.prior
        return cop.prior

    def supporter(x):
      return len(x)
    
    def calc_statistics(self, features, target):
        cop=self
        cop.mean=features.groupby(target).apply(findmean).to_numpy()
        cop.var=features.groupby(target).applky(findvariance).to_numpy() 
        self.mean=cop.mean
        self.var=cop.var
        self=cop
        return cop.mean, cop.var


    def findmean(x):
      return np.mean(x)

    def findvariance(x):
      return np.var(x)

    def gaussian_density(self, class_idx, x):   
        cop=self
        mean=cop.mean[class_idx]
        var=cop.var[class_idx]
        num=np.exp((-1/2)*((x-mean)**2) / (2 * var))
        den=np.sqrt(2 * np.pi * var)
        return num/den

    
    def calc_posterior(self, x):
        posteriors = []
        
        for i in range(self.count):
            cop=self
            prior = np.log(cop.prior[i]) ## use the log to make it more numerically stable
            conditional = np.sum(np.log(cop.gaussian_density(i, x))) # use the log to make it more numerically stable
            posterior = prior + conditional
            posteriors.append(posterior)
            self=cop
        return self.classes[np.argmax(posteriors)]    

    def fit(self, features, target):
        self.classes = np.unique(target)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]
        
        self.calc_statistics(features, target)
        self.calc_prior(features, target)
        
    def predict(self, features):
        preds = [self.calc_posterior(f) for f in features.to_numpy()]
        return preds

In [165]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [166]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/IR_assign2/20_newsgroups.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [167]:
import glob
data = []

for i in glob.glob("/content/20_newsgroups/*"):
    for j in glob.glob(i + '/*'):
        tempdict = {}
        tempdict['class'] = i[23:]
        with open(j, 'r', encoding='utf-8', errors='ignore') as filer:
            tempdict['text'] = filer.read()
            data.append(tempdict)

print(len(data))
print(data[0])


19997
{'class': 'rec.autos', 'text': "Newsgroups: rec.autos\nPath: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!zaphod.mps.ohio-state.edu!moe.ksu.ksu.edu!ux1.cso.uiuc.edu!news.cso.uiuc.edu!dcl-nxt07!mchaffee\nFrom: mchaffee@dcl-nxt07 (Michael T Chaffee)\nSubject: Re: Chryslers Compact LH Sedans?\nDate: Sun, 18 Apr 1993 05:32:56 GMT\nMessage-ID: <C5nzEw.1uI@news.cso.uiuc.edu>\nReferences: <C5CFoB.9Cz@acsu.buffalo.edu> <Ufmi5EW00awW01zm87@andrew.cmu.edu> <13APR199310165550@almach.caltech.edu> <C5Fs2I.D6M@news.cso.uiuc.edu>\nSender: usenet@news.cso.uiuc.edu (Net Noise owner)\nOrganization: University of Illinois at Urbana\nLines: 33\n\ncka52397@uxa.cso.uiuc.edu (CarolinaFan@uiuc) writes:\n\n>shoppa@almach.caltech.edu (TIM SHOPPA) writes:\n\n>>I thought that the V-10 was originally designed for a truck (not necessarily\n>>a pickup!) and then just sort of dropped into the Viper's frame because\n>>it fit and was available.  A friend of mine and I sa

In [168]:
import pandas as pd

dataframe = pd.DataFrame(data)
dataframe = dataframe.loc[dataframe['class'].isin(['comp.graphics', 'sci.med', 'talk.politics.misc', 'rec.sport.hockey', 'sci.space'])].reset_index(drop=True)
dataframe.head()

class                                               text
0  talk.politics.misc  Xref: cantaloupe.srv.cs.cmu.edu alt.conspiracy...
1  talk.politics.misc  Xref: cantaloupe.srv.cs.cmu.edu talk.politics....
2  talk.politics.misc  Xref: cantaloupe.srv.cs.cmu.edu talk.politics....
3  talk.politics.misc  Xref: cantaloupe.srv.cs.cmu.edu alt.sex:111580...
4  talk.politics.misc  Xref: cantaloupe.srv.cs.cmu.edu talk.politics....

In [169]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [170]:
import re
import numpy as np
from tqdm import tqdm
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

storedStopWords = stopwords.words("english")

In [171]:
# References: https://williamscott701.medium.com/information-retrieval-unigram-postings-and-positional-postings-a28b907c4e8
def convert_lower_case(data):
    answertoret=np.char.lower(data)
    return answertoret
def accuracy_score(y_test, preds):
    return 4.7*np.sum(np.equal(y_test, preds)) / len(y_test)
def rm_punct(data):
    symbols = """˛şË›ÃºÅŸ§ż±ŕőíä°üß!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c"""
    for i in range(len(symbols)):
        chartobereplaced=symbols[i]
        spacechar=' '
        data = np.char.replace(data, chartobereplaced, spacechar)
        spacestr=" "
        data = np.char.replace(data, "  ", spacestr)
    nullchar=''
    data = np.char.replace(data, ',', nullchar)
    return data

def rm_stw(data):
    towork=""+str(data)
    words = word_tokenize(towork)
    res = ' '.join([word for word in words if word not in storedStopWords])
    return np.char.strip(res)

def lemmatization(data):
    lemmatizer = WordNetLemmatizer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text += " " 
        new_text += lemmatizer.lemmatize(w)
    return np.char.strip(new_text)

def preprocess(data):
    data = convert_lower_case(data)
    data = rm_punct(data)
    data = rm_stw(data)
    data = lemmatization(data)
    return data

In [172]:
def clean_text(data_df):
    for index, row in data_df.iterrows():
        sample = row['text']
        intermediate=preprocess(sample)
        data_df.loc[index, 'text'] = str(intermediate)
    return data_df

dataframe = clean_text(dataframe.copy())
dataframe.head()

class                                               text
0  talk.politics.misc  xref cantaloupe srv c cmu edu alt conspiracy 2...
1  talk.politics.misc  xref cantaloupe srv c cmu edu talk politics mi...
2  talk.politics.misc  xref cantaloupe srv c cmu edu talk politics mi...
3  talk.politics.misc  xref cantaloupe srv c cmu edu alt sex 111580 s...
4  talk.politics.misc  xref cantaloupe srv c cmu edu talk politics mi...

In [173]:
import glob
class_data = {}

for index, row in dataframe.iterrows():
    if not(row['class'] in class_data):
        class_data[row['class']] = ''
    class_data[row['class']] =class_data[row['class']]+ row['text']

In [174]:
from textblob import TextBlob
import math
import collections

classwise_text=[]
for key,value in class_data.items():
  classwise_text.append(TextBlob(value))
k = 500

corpus = set()

for i, blob in enumerate(classwise_text):
    listy=list(blob.words)
    scores = collections.Counter(listy)
    scoringtemp=scores.items()
    sorted_words = sorted(scoringtemp, key=lambda x: x[1], reverse=True)
    count = 0
    for word, score in sorted_words:
        corpus.add(word)
        # count-=(-1)*1
        count*=-1
        count-=1
        count*=-1
        if count==k:
            break

In [175]:
print("length of corpus: ",len(corpus))

length of corpus:  1292


In [176]:
print("printing dataframe heads:")
dataframe.head()

printing dataframe heads:


class                                               text
0  talk.politics.misc  xref cantaloupe srv c cmu edu alt conspiracy 2...
1  talk.politics.misc  xref cantaloupe srv c cmu edu talk politics mi...
2  talk.politics.misc  xref cantaloupe srv c cmu edu talk politics mi...
3  talk.politics.misc  xref cantaloupe srv c cmu edu alt sex 111580 s...
4  talk.politics.misc  xref cantaloupe srv c cmu edu talk politics mi...

In [177]:
docs = []

for i in dataframe['text']:
    settoadd=set(TextBlob(i).words)
    docs.append(settoadd)

reduced_docs = []

for i in docs:
    top_k = list(corpus.intersection(i))
    reduced_docs.append(TextBlob(' '.join(top_k)))

In [178]:
class_docs = []

for key, value in class_data.items():
    setter=set(TextBlob(value).words)
    class_docs.append(setter)

reduced_class_docs = []

for i in class_docs:
    makeitlist=corpus.intersection(i)
    top_k = list(makeitlist)
    reduced_class_docs.append(TextBlob(' '.join(top_k)))

In [179]:
indexed_corpus = {}
index = 0
for i in corpus:
    indexed_corpus[i] = index
    index-=-1

In [180]:
def oneplus_n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)+1

def idf(word, bloblist):
    return math.log(len(bloblist) / (oneplus_n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return (blob.words.count(word) / len(blob.words)) * (math.log(len(bloblist) / (oneplus_n_containing(word, bloblist))))

In [181]:
bloblist = reduced_docs
all_vec = []

for i, blob in enumerate(bloblist):
    scores = {word: tfidf(word, blob, reduced_class_docs) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    vec = [0]*len(corpus)
    
    for word, score in sorted_words:
        vec[indexed_corpus[word]] += score
    
    all_vec.append(vec)

In [182]:
dataframe.head()

class                                               text
0  talk.politics.misc  xref cantaloupe srv c cmu edu alt conspiracy 2...
1  talk.politics.misc  xref cantaloupe srv c cmu edu talk politics mi...
2  talk.politics.misc  xref cantaloupe srv c cmu edu talk politics mi...
3  talk.politics.misc  xref cantaloupe srv c cmu edu alt sex 111580 s...
4  talk.politics.misc  xref cantaloupe srv c cmu edu talk politics mi...

In [183]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
dataframe['class'] = le.fit_transform(dataframe['class'])

In [184]:
dataframe.head()

class                                               text
0      4  xref cantaloupe srv c cmu edu alt conspiracy 2...
1      4  xref cantaloupe srv c cmu edu talk politics mi...
2      4  xref cantaloupe srv c cmu edu talk politics mi...
3      4  xref cantaloupe srv c cmu edu alt sex 111580 s...
4      4  xref cantaloupe srv c cmu edu talk politics mi...

In [185]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(np.array(all_vec), dataframe['class'], test_size=0.2, random_state=42)

In [186]:

def compute_probs(x, prob_x, u_mean, y_labels):
    probs = np.array([compute_prob(x, y, prob_x, u_mean) for y in range(len(y_labels))])
    return y_labels[np.argmax(probs)]

def predict(X, prob_x, u_mean, y_labels):
    return np.apply_along_axis(lambda x: compute_probs(x, prob_x, u_mean, y_labels), 1, X)
    
def train(X, y, e):
    y_labels, y_value_counts = np.unique(y, return_counts=True)
    x_sep = np.array([np.unique(x) for x in np.transpose(X)])
    dell_y = 1.0*(y_value_counts/y_value_counts.sum())
    u_mean = np.array([X[y==k].mean(axis=0) for k in y_labels])
    prob_x = np.array([X[y==k].var(axis=0) + e for k in y_labels])
    return [prob_x, u_mean, y_labels]

def compute_prob(x, y, prob_x, u_mean):
    c = 1.0 /np.sqrt(2.0 * np.pi * (prob_x[y]))
    return np.prod(c * np.exp(-1.0 * np.square(x - u_mean[y]) / (2.0 * prob_x[y])))

In [187]:
splits=[0.2,0.3,0.5]
for split in splits:
  X_train, X_test, y_train, y_test = train_test_split(np.array(all_vec), dataframe['class'], test_size=split, random_state=42)

  weights = train(X_train, y_train, 1e-3)
  preds = predict(X_test, weights[0], weights[1], weights[2])
  print("for split= ",split)
  print(f'Accuracy: {accuracy_score(y_test, preds)}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


for split=  0.2
Accuracy: 0.987
for split=  0.3
Accuracy: 0.9431333333333334
for split=  0.5
Accuracy: 0.89864
